In [2]:
import os
import subprocess
import sys
import shutil
import logging
import datetime
import glob

In [3]:
log_filename = f"processing_errors_{datetime.datetime.now().strftime('%Y%m%d_%H%M%S')}.log"
logging.basicConfig(
    filename=log_filename,
    level=logging.ERROR,
    format='%(asctime)s - %(levelname)s - %(message)s',
    datefmt='%Y-%m-%d %H:%M:%S'
)

In [4]:
def run_command(command, check=False):
    """Run a command and return its output."""
    print(f"Executing: {command}")
    try:
        result = subprocess.run(
            command, 
            check=check, 
            shell=True, 
            stdout=subprocess.PIPE, 
            stderr=subprocess.PIPE,
            universal_newlines=True
        )
        print(result.stdout)
        if result.stderr:
            print(f"Error output: {result.stderr}")
        return result
    except subprocess.CalledProcessError as e:
        error_msg = f"Command failed with return code {e.returncode}: {command}\nError: {e.stderr}"
        print(error_msg)
        # Логирование ошибки в файл
        logging.error(error_msg)
        return False, e

In [5]:
def copy_and_remove(source, destination):
    """Копирует файл из источника в назначение и удаляет исходный файл"""
    try:
        if os.path.exists(source):
            shutil.copy2(source, destination)
            print(f"Copied {source} to {destination}")
            os.remove(source)
            print(f"Removed {source}")
            return True
        else:
            print(f"Source file {source} does not exist, skipping")
            return False
    except Exception as e:
        error_msg = f"Error copying/removing file {source} to {destination}: {str(e)}"
        print(error_msg)
        logging.error(error_msg)
        return False

In [9]:
def cleanup_results_directories():
    print("Очистка директорий результатов, оставляем только info.json...")
    
    results_dir = './results'
    if not os.path.exists(results_dir):
        print(f"Директория {results_dir} не существует, очистка не требуется")
        return
    
    # Перебираем все поддиректории
    for root, dirs, files in os.walk(results_dir):
        # Пропускаем корневую директорию
        if root == results_dir:
            continue
        
        # Проверяем каждый файл
        for file in files:
            file_path = os.path.join(root, file)
            
            # Если это не info.json - удаляем
            if file != 'info.json':
                try:
                    os.remove(file_path)
                    print(f"Удален файл: {file_path}")
                except Exception as e:
                    error_msg = f"Не удалось удалить файл {file_path}: {str(e)}"
                    print(error_msg)
                    logging.error(error_msg)

In [15]:
def find_directories_without_info_json():
    print("Поиск директорий без файла info.json...")
    
    results_dir = './results'
    if not os.path.exists(results_dir):
        print(f"Директория {results_dir} не существует")
        return []
    
    directories_without_info = []
    
    # Перебираем все поддиректории в results
    for root, dirs, files in os.walk(results_dir):
        # Пропускаем корневую директорию и директории верхнего уровня (1d, 2d, 3d, 4d)
        if root == results_dir or os.path.dirname(root) == results_dir:
            continue
        
        # Проверяем наличие info.json
        if 'info.json' not in files:
            directories_without_info.append(root)
            print(f"Директория без info.json: {root}")
    
    # Записываем список в файл
    # report_filename = 'directories_without_info.txt'
    # with open(report_filename, 'w') as f:
    #     f.write(f"Всего директорий без info.json: {len(directories_without_info)}\n\n")
    #     for directory in directories_without_info:
    #         f.write(f"{directory}\n")
    
    # print(f"Список директорий без info.json сохранен в файле {report_filename}")
    print(f"Всего найдено директорий без info.json: {len(directories_without_info)}")
    
    return directories_without_info

In [6]:
sapfor_dir = '../SAPFOR/_bin/Release/Sapfor_F.exe'
source_dir = './sources'

dimensions = ['1d', '2d', '3d', '4d']

In [ ]:
# Использование os.makedirs() вместо команды mkdir
if not os.path.exists('./results'):
    os.makedirs('./results')


# Обработка всех размерностей
for dim in dimensions:
    # Создаем папку для размерности, если она не существует
    results_dim_path = os.path.join('./results', dim)
    if not os.path.exists(results_dim_path):
        os.makedirs(results_dim_path)
    
    # Получаем список файлов в исходной директории
    dim_dir = os.path.join(source_dir, dim)
    files = os.listdir(dim_dir)
    
    for file in files:
        if file.endswith('.f'):
            file_name = file.split('.')[0]
            
            # Создание директории для файла
            file_dir_path = os.path.join(results_dim_path, file_name)
            if not os.path.exists(file_dir_path):
                os.makedirs(file_dir_path)
            
            # Исходный и целевой пути для копирования
            source_file = os.path.join(dim_dir, file)
            dest_file = os.path.join(file_dir_path, file)
            
            # Копирование файла с помощью shutil вместо команды cp
            if not os.path.exists(dest_file):
                try:
                    shutil.copy2(source_file, dest_file)
                    print(f"Copied {source_file} to {dest_file}")
                except Exception as e:
                    error_msg = f"Error copying file {source_file} to {dest_file}: {str(e)}"
                    print(error_msg)
                    logging.error(error_msg)
            
            try:
                # Выполнение команд для обработки файла
                run_command(f'gfortran -O3 -g -fprofile-arcs -ftest-coverage "{dest_file}" -o "{os.path.join(file_dir_path, file_name)}"')
                run_command(f'"{os.path.join(file_dir_path, file_name)}"')
                run_command(f'gcov -b "{dest_file}"')

                # Копирование gcov-файла и удаление оригинала
                gcov_file = f'./{file}.gcov'
                gcov_dest = os.path.join(file_dir_path, f'{file}.gcov')
                copy_and_remove(gcov_file, gcov_dest)

                run_command(f'"{sapfor_dir}" -parse -spf "{dest_file}"')
                run_command(f'"{sapfor_dir}" -passN GET_STATS_FOR_PREDICTOR -keepDVM')

                # Копирование info.json и удаление оригинала
                info_file = './info.json'
                info_dest = os.path.join(file_dir_path, 'info.json')
                copy_and_remove(info_file, info_dest)
            except Exception as e:
                error_msg = f"Error processing {file}: {str(e)}"
                print(error_msg)
                logging.error(error_msg)
                continue

temp_files_to_clean = [
    '*.gcda', '*.gcno', '*.gcov', '*.dep', '*.proj', '*.mod'
]

for pattern in temp_files_to_clean:
    try:
        files = glob.glob(pattern)
        for file in files:
            try:
                os.remove(file)
                print(f"Cleaned up temporary file: {file}")
            except Exception as e:
                error_msg = f"Failed to remove temp file {file}: {str(e)}"
                print(error_msg)
                logging.error(error_msg)
    except Exception as e:
        error_msg = f"Error during cleanup of pattern {pattern}: {str(e)}"
        print(error_msg)
        logging.error(error_msg)

print(f"Процесс завершен. Журнал ошибок сохранен в файле: {log_filename}")

Copied ./sources\1d\acr11.f to ./results\1d\acr11\acr11.f
Executing: gfortran -O3 -g -fprofile-arcs -ftest-coverage "./results\1d\acr11\acr11.f" -o "./results\1d\acr11\acr11"

Executing: "./results\1d\acr11\acr11"
 ===START OF ACR11========================
 ACR1101  -  complete
 ACR1102  -  complete
 ACR1103  -  complete
 ACR1104  -  complete
 ACR1105  -  complete
 ACR1106  -  complete
 ACR1107  -  complete
 ACR1108  -  complete
 ACR1109  -  complete
 ACR1110  -  complete
 === END OF ACR11 ========================= 

Executing: gcov -b "./results\1d\acr11\acr11.f"
File 'results/1d/acr11/acr11.f'
Lines executed:89.32% of 234
Branches executed:99.17% of 242
Taken at least once:74.38% of 242
Calls executed:37.93% of 87
Creating 'acr11.f.gcov'

Lines executed:89.32% of 234

Copied ./acr11.f.gcov to ./results\1d\acr11\acr11.f.gcov
Removed ./acr11.f.gcov
Executing: "../SAPFOR/_bin/Release/Sapfor_F.exe" -parse -spf "./results\1d\acr11\acr11.f"
SAPFOR version is 2409, build date: Apr 17 2025 1

In [10]:
cleanup_results_directories()

Очистка директорий результатов, оставляем только info.json...
Удален файл: ./results\1d\acr11\acr11.exe
Удален файл: ./results\1d\acr11\acr11.f
Удален файл: ./results\1d\acr11\acr11.f.dep
Удален файл: ./results\1d\acr11\acr11.f.gcov
Удален файл: ./results\1d\acr11\acr11.gcda
Удален файл: ./results\1d\acr11\acr11.gcno
Удален файл: ./results\1d\acr12\acr12.DVMH.f
Удален файл: ./results\1d\acr12\acr12.DVMH.f.dep
Удален файл: ./results\1d\acr12\acr12.exe
Удален файл: ./results\1d\acr12\acr12.f
Удален файл: ./results\1d\acr12\acr12.f.dep
Удален файл: ./results\1d\acr12\acr12.f.gcov
Удален файл: ./results\1d\acr12\acr12.gcda
Удален файл: ./results\1d\acr12\acr12.gcno
Удален файл: ./results\1d\align11\align11.exe
Удален файл: ./results\1d\align11\align11.f
Удален файл: ./results\1d\align11\align11.f.dep
Удален файл: ./results\1d\align11\align11.f.gcov
Удален файл: ./results\1d\align11\align11.gcda
Удален файл: ./results\1d\align11\align11.gcno
Удален файл: ./results\1d\align12\align12.exe
Уда

In [16]:
directories_without_info = find_directories_without_info_json()

Поиск директорий без файла info.json...
Директория без info.json: ./results\1d\align11
Директория без info.json: ./results\1d\alignfloat11
Директория без info.json: ./results\1d\distrfloat1
Директория без info.json: ./results\1d\f2c_do
Директория без info.json: ./results\1d\f2c_math_intel
Директория без info.json: ./results\1d\f2c_select
Директория без info.json: ./results\1d\parallel1
Директория без info.json: ./results\1d\parallelNoOn1
Директория без info.json: ./results\1d\parallelNoOn12
Директория без info.json: ./results\1d\parallelNoOn13
Директория без info.json: ./results\1d\paralplus12
Директория без info.json: ./results\1d\paralplus14
Директория без info.json: ./results\1d\realign11
Директория без info.json: ./results\1d\taskst11
Директория без info.json: ./results\1d\taskst12
Директория без info.json: ./results\1d\templ1
Директория без info.json: ./results\2d\align22
Директория без info.json: ./results\2d\align24
Директория без info.json: ./results\2d\copy21
Директория без in